#4-8 クラスタリング

##概要
**クラスタリングは、データなどの集合体を機能やカテゴリごとに分けて集める手法です。**

## クラスタリング(教師あり学習)
教師あり学習におけるクラスタリングとして、k 近傍法
（k-Nearest Neighbor :k-NN）があります。K近傍法と、k-Means法（K-平均法）とは別物だということに注意してください。

###k 近傍法（k-NN法）

**k 近傍法は、あらかじめクラス分けされた教師データをもとに、新しいデータのクラスを最も近いk 個データのクラスから多数決で分類する手法です。**

オブジェクトの分類を、その近傍のオブジェクト群の投票によって決定します。
図は、K=3 及び k=5 の例です。

あらかじめ与えられたデータは、赤い丸と、黒い四角になります。ここに新たなデー
タである灰色の三角が、丸と四角のどちらに分類されるかを予測します。k= 3 で予
測する場合は、赤い丸が2 つ、黒い四角が1 つなので、灰色の三角は赤い丸に分類さ
れます。一方、k= 5 の場合は、赤い丸が2 つ、黒い四角が3 つなので、灰色の三角
は黒い四角に分類されることになります。

K 近傍法のメリットは、アルゴリズムが単純でわかりやすいことです。欠点は、次
元の数が大きいと類似度の距離が測りにくく、適用が難しくなることです。

アイテムのレコメンデーションや、機械の故障などの異常値検出に使われます。

<img src="https://github.com/t-date/DataScience/blob/master/fig/04_08_01.jpg?raw=true" width="280px">

4-1で使用したOECDとGDPのデータを使用しk近傍法を実装してみましょう。

In [0]:
#データの取り込み
!wget -nc /content 'https://raw.githubusercontent.com/t-date/DataScience/master/data/oecd_bli_2015.csv'
!wget -nc /content 'https://raw.githubusercontent.com/t-date/DataScience/master/data/gdp_per_capita.csv'

In [0]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
# データをロードする
oecd_bli = pd.read_csv("oecd_bli_2015.csv", thousands=",")
gdp_per_capita =pd.read_csv("gdp_per_capita.csv",thousands=",",delimiter="\t", encoding="latin1", na_values="n/a")

In [0]:
def prepare_country_stats(oecd_bli, gdp_per_capita):
    oecd_bli = oecd_bli[oecd_bli["INEQUALITY"]=="TOT"]
    oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
    gdp_per_capita.rename(columns={"2015": "GDP per capita"}, inplace=True)
    gdp_per_capita.set_index("Country", inplace=True)
    full_country_stats = pd.merge(left=oecd_bli, right=gdp_per_capita,
                                  left_index=True, right_index=True)
    full_country_stats.sort_values(by="GDP per capita", inplace=True)
    remove_indices = [0, 1, 6, 8, 33, 34, 35]
    keep_indices = list(set(range(36)) - set(remove_indices))
    return full_country_stats[["GDP per capita", 'Life satisfaction']].iloc[keep_indices]

In [0]:
# データを準備する
country_stats = prepare_country_stats(oecd_bli, gdp_per_capita)
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]

In [0]:
# データを可視化する
country_stats.plot(kind="scatter", x="GDP per capita", y="Life satisfaction")
plt.show()

k 近傍法モデルを実装してみます。

In [0]:
# k 近傍法モデルを選択する
from sklearn import neighbors as nb
model = nb.KNeighborsRegressor(n_neighbors=3)

In [0]:
# モデルを訓練する
model.fit(X, y)

In [0]:
# キプロスの例から予測を行う
X_new = [[22587]] # キプロスの1 人あたりGDP
print(model.predict(X_new)) # 出力[[ 5.96242338]]

予測モデルを引いてみます。

In [0]:
country_stats.plot(kind="scatter", x="GDP per capita", y="Life satisfaction")
plt.plot(X, model.predict(X))

## クラスタリング(教師なし学習)
教師なし学習で典型的な例は、与えられたデータをいくつかのクラスターに分類するクラスタリングです。

教師なし学習のクラスタリングで広く用いられる手法としては、
K-Means 法（またはK-平均法）があります。K-Means法と、K-近傍法(K-NN法)とは別物だということに注意してください。

###K-Means法
**データをk個のクラスタに分け、各クラスタの重心に一番近い点をそのクラスタに分類しなおすということを繰り返して、データをクラスタリングする手法です。**

シンプルなアルゴリズムのため、広く用いられています。

最適なkを見積もる方法としては、SSE（残差平方和）の減少量を見るエルボー法、クラスタ内のデータの凝集度を見るシルエット法が知られています。

k-meansクラスタリングは、最も単純で最も広く用いられているクラスタリングアルゴリズムであり、このアルゴリズムは、データのある領域を代表するようなクラスタ重心を見つけようとします。

このアルゴリズムは次の2つのステップを繰り返します。個々のデータポイントを最寄りのクラスタ重心に割り当てる。次に、個々のクラスタ重心をその点に割り当てられたデータポイントの平均に設定する。データポイントの割り当てが変化しなくなったら、アルゴリズムは終了します。
※下図はイメージで星印は各クラスタの重心

<img src="https://github.com/t-date/DataScience/blob/master/fig/04_08_02.jpg?raw=true" width="280px">

scikit-learnでは数多くのクラスタリングアルゴリズムが提供されていますが、理解しやすく最も単純なアルゴリズムはk 平均法（k-means）クラスタリングと呼ばれ、sklearn.cluster.KMeans で実装されています。標準的なモジュールを最初にインポートします。

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # プロットのスタイルを決める
import numpy as np

k-Means法のアルゴリズムは、ラベル付けされていない多次元データセット内で与えられた数のクラスタを探します。これは、最適なクラスタリングはどのように見えるかという、次のような単純な概念を使用します。

*   「クラスタの中心」は、クラスタに属するすべてのポイントの算術平均である。
*   各ポイントは、他のクラスタ中心よりも自分の属するクラスタの中心に近い。

この2 つの前提が、k-Means法モデルの基礎となりますまずは単純なデータセットにk 平均法を適用した結果を見てみましょう。

まず、4つの異なる集団からなる2 次元のデータセットを生成します。これが教師なし学習アルゴリズムであることを強調するために、ラベルをプロットには含めません。



In [0]:
from sklearn.datasets.samples_generator import make_blobs
X, y_true = make_blobs(n_samples=300, centers=4,
cluster_std=0.60, random_state=0)
plt.scatter(X[:, 0], X[:, 1], s=50);

見た目では、4 つのクラスタを識別するのは比較的簡単です。k 平均法アルゴリズムはこれを自動的に行います。scikit-learnは典型的な推定器APIを提供します。

In [0]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)
kmeans.fit(X)
y_kmeans = kmeans.predict(X)

得られたラベルで色付けしたデータをプロットして、結果を可視化しましょう。また、k-Means法推定器によって決定されたクラスタの中心もプロットします。

In [0]:
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='viridis')

k-Means法アルゴリズム（少なくともこのような単純なケースでは）は見た目でクラスタを識別する方法とほぼ同じ方法で各点をクラスタに割り当てます。しかし、このアルゴリズムがいかにしてクラスタを素早く見つけ出すのか疑問に思うかもしれません。結局、クラスタ割り当て可能な組み合わ
せの数はデータポイントの数に対して指数関数的に増加するため、網羅的な検索は非常にコストが
かかります。幸いなことに、網羅的な検索は必要ありません。代わりに、k-Means法の典型的なアプ
ローチは、期待値最大化法と呼ばれる直感的な反復アプローチを用います。

###期待値最大化法k-Means法

期待値最大化法（E-M：Expectation-maximization）は、データサイエンスのさまざまな状況で使
用される強力なアルゴリズムです。k 平均法は、このアルゴリズムの特に簡単でわかりやすい応用
の1つです。簡単に説明しましょう。期待値最大化法は以下の手順で構成されています。
1.   クラスタ中心を推測する
2.   収束するまで繰り返す
> a. Eステップ：各ポイントを最近傍のクラスタ中心に割り当てる   
> b. Mステップ：各ポイントの平均をクラスタ中心に設定する

まず、各ポイントがどのクラスタに属するかの予想を更新する「Eステップ」または「期待値（Expectation）ステップ」を実行します。次に、ここでの場合はクラスタ中心を定義するフィットネス関数を最大化する「Mステップ」または「最大化（Maximization）ステップ」を実行します。この最大化は各クラスタに含まれるデータの単純な平均を取ることで行われます。

このアルゴリズムに関する論文は膨大に存在しますが、以下のように要約することができます。
つまり典型的な状況下では、E ステップおよびMステップ繰り返しは、常にクラスタ特性のより
良い推定をもたらします。






In [0]:
from sklearn.metrics import pairwise_distances_argmin
def find_clusters(X, n_clusters, rseed=2):
  # 1. ランダムにクラスタを選択
  rng = np.random.RandomState(rseed)
  i = rng.permutation(X.shape[0])[:n_clusters]
  centers = X[i]
  while True:
    # 2a. 最近傍のクラスタでラベル付けを行う
    labels = pairwise_distances_argmin(X, centers)
  
  
    # 2b. ポイントの平均からクラスタ中心決定する
    new_centers = np.array([X[labels == i].mean(0) for i in range(n_clusters)])
    
    # 2c. 収束チェック
    if np.all(centers == new_centers):
      break
    
    centers = new_centers
  
  return centers, labels

centers, labels = find_clusters(X, 4)
plt.scatter(X[:, 0], X[:, 1], c=labels, s=50, cmap='viridis')

十分にテストされた多くの実装は舞台裏でさらに多くの作業を行いますが、ここで示した関数は期待値最大化法アプローチの要点を示しています。